# Data Cleaning and Alignment

In this notebook, basic data cleaning steps are applied to the raw market datasets.

The objective is to ensure consistent timestamps, handle missing values, and prepare the data for feature engineering.

Given the academic nature of this project, the cleaning approach focuses on simplicity and correctness rather than exchange-level precision.


In [1]:
import pandas as pd
import numpy as np


In [3]:
spot = pd.read_csv("../data/nifty_spot_1min.csv")
print(spot.head())



                  date     open     high      low    close  volume
0  2015-01-09 09:15:00  8285.45  8295.90  8285.45  8292.10       0
1  2015-01-09 09:16:00  8292.60  8293.60  8287.20  8288.15       0
2  2015-01-09 09:17:00  8287.40  8293.90  8287.40  8293.90       0
3  2015-01-09 09:18:00  8294.25  8300.65  8293.90  8300.65       0
4  2015-01-09 09:19:00  8300.60  8301.30  8298.75  8301.20       0


In [6]:
# Load spot data
spot = pd.read_csv("../data/nifty_spot_1min.csv")

# Rename column
spot.rename(columns={"date": "timestamp"}, inplace=True)

# Convert timestamp
spot["timestamp"] = pd.to_datetime(spot["timestamp"])

spot.head()


,timestamp,open,high,low,close,volume
0,2015-01-09 09:15:00,8285.45,8295.90,8285.45,8292.10,0
1,2015-01-09 09:16:00,8292.60,8293.60,8287.20,8288.15,0
2,2015-01-09 09:17:00,8287.40,8293.90,8287.40,8293.90,0
3,2015-01-09 09:18:00,8294.25,8300.65,8293.90,8300.65,0
4,2015-01-09 09:19:00,8300.60,8301.30,8298.75,8301.20,0


In [7]:
spot_5min = spot.set_index("timestamp").resample("5T").agg({
    "open": "first",
    "high": "max",
    "low": "min",
    "close": "last",
    "volume": "sum"
}).dropna().reset_index()

spot_5min.head()


C:\Users\navin\AppData\Local\Temp\ipykernel_5972\3811305811.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  spot_5min = spot.set_index("timestamp").resample("5T").agg({


,timestamp,open,high,low,close,volume
0,2015-01-09 09:15:00,8285.45,8301.30,8285.45,8301.20,0
1,2015-01-09 09:20:00,8300.50,8303.00,8293.25,8301.00,0
2,2015-01-09 09:25:00,8301.65,8302.55,8286.80,8294.15,0
3,2015-01-09 09:30:00,8294.10,8295.75,8280.65,8288.50,0
4,2015-01-09 09:35:00,8289.10,8290.45,8278.00,8283.45,0


## Notes and Assumptions

- Exact futures rollover handling is simplified.
- Options data is restricted to ATM strikes to reduce complexity.
- The focus is on feature behavior rather than perfect market replication.


In [8]:
spot_5min.to_csv("../data/nifty_spot_5min.csv", index=False)


In [9]:
# Create synthetic futures data based on spot

futures = spot_5min.copy()

# Add small futures basis (0.05%)
futures["close"] = futures["close"] * (1 + 0.0005)
futures["open"] = futures["open"] * (1 + 0.0005)
futures["high"] = futures["high"] * (1 + 0.0005)
futures["low"] = futures["low"] * (1 + 0.0005)

# Simulate open interest
futures["open_interest"] = np.random.randint(
    100000, 200000, size=len(futures)
)

futures.head()


,timestamp,open,high,low,close,volume,open_interest
0,2015-01-09 09:15:00,8289.592725,8305.450650,8289.592725,8305.350600,0,131916
1,2015-01-09 09:20:00,8304.650250,8307.151500,8297.396625,8305.150500,0,188438
2,2015-01-09 09:25:00,8305.800825,8306.701275,8290.943400,8298.297075,0,199035
3,2015-01-09 09:30:00,8298.247050,8299.897875,8284.790325,8292.644250,0,182187
4,2015-01-09 09:35:00,8293.244550,8294.595225,8282.139000,8287.591725,0,102118


In [10]:
futures.to_csv("../data/nifty_futures_5min.csv", index=False)


In [11]:
options_list = []

for _, row in spot_5min.iterrows():
    timestamp = row["timestamp"]
    spot_price = row["close"]
    
    atm_strike = round(spot_price / 50) * 50
    
    for option_type in ["CE", "PE"]:
        options_list.append({
            "timestamp": timestamp,
            "strike": atm_strike,
            "option_type": option_type,
            "ltp": abs(spot_price - atm_strike) + np.random.uniform(10, 30),
            "iv": np.random.uniform(0.15, 0.25),
            "open_interest": np.random.randint(5000, 20000),
            "volume": np.random.randint(100, 1000)
        })

options = pd.DataFrame(options_list)
options.head()


,timestamp,strike,option_type,ltp,iv,open_interest,volume
0,2015-01-09 09:15:00,8300,CE,23.673277,0.207928,17007,269
1,2015-01-09 09:15:00,8300,PE,18.418679,0.227167,11444,789
2,2015-01-09 09:20:00,8300,CE,17.207454,0.208698,8989,611
3,2015-01-09 09:20:00,8300,PE,30.285380,0.210357,10925,575
4,2015-01-09 09:25:00,8300,CE,21.861982,0.181469,5750,797


In [12]:
options.to_csv("../data/nifty_options_5min.csv", index=False)


## Data Construction Notes

- Futures prices are simulated from spot prices with a small basis to maintain realistic alignment.
- Options data is limited to ATM Call and Put options.
- Implied volatility values are approximated to enable feature engineering and regime detection.
- The objective is to study feature behavior rather than replicate exact market prices.
